# **Feature Engeneering - Bureau Balance**

## **Lendo dados com Spark**

### **Importar Bibliotecas**

In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1701548887197_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
import os
import sys
import pytz
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Importar tabela 'bureau_balance'.**

In [4]:
spark = SparkSession.builder.appName('Feature Engineering').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1701548887197_0001/container_1701548887197_0001_01_000001/pyspark.zip/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

In [6]:
file='bureau_balance.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
S3_DATA_SOURCE_PATH = 'pod-academy-analise-de-credito-para-fintech/dados'
S3_DATA_OUTPUT_PATH = 'pod-academy-analise-de-credito-para-fintech/feature-engineering/01-Bureau'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
output = 'abt_bureau_balance'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
full_path_source = 's3://{bkt}/{file}'.format(bkt=S3_DATA_SOURCE_PATH,file=file)
print(full_path_source)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/dados/bureau_balance.csv

In [15]:
bureau_balance_00 = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_source)
bureau_balance_00.registerTempTable("bureau_balance_00")
bureau_balance_00.cache()
qtd=bureau_balance_00.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 27299925

In [16]:
bureau_balance_00.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|     5715448|             0|     C|
|     5715448|            -1|     C|
|     5715448|            -2|     C|
|     5715448|            -3|     C|
|     5715448|            -4|     C|
|     5715448|            -5|     C|
|     5715448|            -6|     C|
|     5715448|            -7|     C|
|     5715448|            -8|     C|
|     5715448|            -9|     0|
|     5715448|           -10|     0|
|     5715448|           -11|     X|
|     5715448|           -12|     X|
|     5715448|           -13|     X|
|     5715448|           -14|     0|
|     5715448|           -15|     0|
|     5715448|           -16|     0|
|     5715448|           -17|     0|
|     5715448|           -18|     0|
|     5715448|           -19|     0|
+------------+--------------+------+
only showing top 20 rows

In [17]:
bureau_balance_00.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- SK_ID_BUREAU: string (nullable = true)
 |-- MONTHS_BALANCE: string (nullable = true)
 |-- STATUS: string (nullable = true)

# **Criação de variáveis preditivas**

## **bureau_balance**

Primeiro iremos criar variáveis preditivas na tabela "bureau_balance" para acrescentar essas variáveis na tabela "bureau".

Após essa união das duas tabelas, iremos realizar a criação de variáveis preditivas na tabela "bureau".

In [18]:
# Definir funções de agregações desejadas
agg = ['SUM', 'MAX', 'MIN', 'AVG']

var_categoricas = ['STATUS']
var_numericas = ['MONTHS_BALANCE']

# Definir flags de meses
meses_flag = ['ultimos_3_meses_flag', 'ultimos_6_meses_flag', 'ultimos_9_meses_flag', 'ultimos_12_meses_flag']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
bureau_balance_00.createOrReplaceTempView('bureau_balance_00')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## **Criando flags de janela para histórico:**

- últimos 3 meses

- últimos 6 meses

- últimos 12 meses

In [20]:
bureau_balance_01 = spark.sql("""
SELECT
    *,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_BUREAU`) - 3 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_BUREAU`) THEN 1
        ELSE 0
    END AS ultimos_3_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_BUREAU`) - 6 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_BUREAU`) THEN 1
        ELSE 0
    END AS ultimos_6_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_BUREAU`) - 6 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_BUREAU`) THEN 1
        ELSE 0
    END AS ultimos_9_meses_flag,
    CASE
        WHEN MONTHS_BALANCE BETWEEN MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_BUREAU`) - 12 AND MAX(MONTHS_BALANCE) OVER (PARTITION BY `SK_ID_BUREAU`) THEN 1
        ELSE 0
    END AS ultimos_12_meses_flag
FROM bureau_balance_00
ORDER BY `SK_ID_BUREAU`, MONTHS_BALANCE;
""")

bureau_balance_01.createOrReplaceTempView("bureau_balance_01")
bureau_balance_01.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27299925

In [21]:
bureau_balance_01.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+------+--------------------+--------------------+--------------------+---------------------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|ultimos_3_meses_flag|ultimos_6_meses_flag|ultimos_9_meses_flag|ultimos_12_meses_flag|
+------------+--------------+------+--------------------+--------------------+--------------------+---------------------+
|     5001709|            -1|     C|                   1|                   1|                   1|                    1|
|     5001709|           -10|     C|                   0|                   0|                   0|                    1|
|     5001709|           -11|     C|                   0|                   0|                   0|                    1|
|     5001709|           -12|     C|                   0|                   0|                   0|                    1|
|     5001709|           -13|     C|                   0|                   0|                   0|                    0|
|     5001709|          

## **Criando variáveis explicativas de primeira camada**

- usar média, mínimo, soma, máximo
- considerar variáveis categóricas e variáveis numéricas

### **Gerar variáveis.**

In [23]:
# Importar SparkSession
from pyspark.sql import SparkSession

# Criar uma sessão do Spark
spark = SparkSession.builder.appName("AdaptacaoCodigo").getOrCreate()

df_original_spark = bureau_balance_01

# Inicializar o contador
contador_resultados = 0

# Criar lista para armazenar os resultados
resultados = []

for ag in agg:
    for var_cat in var_categoricas:
        # Filtrar valores distintos não nulos
        distinct_values = df_original_spark.select(var_cat).distinct().na.drop(subset=[var_cat]).rdd.flatMap(lambda x: x).collect()

        for dom_cat_value in distinct_values:
            for var_numerica in var_numericas:
                resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_{var_numerica.lower()[:5]}_{var_cat.lower()[:4]}_{dom_cat_value.lower()[:3].replace(' ', '_')},"

                # Adicionar o resultado à lista
                resultados.append(resultado)

                for mes_flag in meses_flag:
                    resultado = f"ROUND({ag}(CASE WHEN {var_cat} = '{dom_cat_value}' AND {mes_flag} = 1 THEN {var_numerica} ELSE NULL END), 2) as {ag.lower()}_U{mes_flag.split('_')[1]}M_{var_numerica.lower()[:5]}_{var_cat.lower()[:4]}_{dom_cat_value.lower()[:3].replace(' ', '_')},"

                    # Adicionar o resultado à lista
                    resultados.append(resultado)

                    # Incrementar o contador
                    contador_resultados += 1

# Printar o total de combinações
print(f"Total de combinações: {contador_resultados}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total de combina??es: 128

In [24]:
# Inicializa a lista para armazenar os resultados
resultados_formatados = []

# Se há pelo menos um resultado na lista
if resultados:
    # Remove a vírgula do último item
    resultados[-1] = resultados[-1].rstrip(',')

    # Adiciona os resultados formatados à lista
    resultados_formatados.extend(resultados)

    # Une a lista em uma string usando uma quebra de linha como separador
    resultados_str = '\n'.join(resultados_formatados)
    # print(resultados_str)
else:
    resultados_str = "A lista de resultados está vazia."

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### **Colar variáveis criadas no código PySpark-SQL.**

In [25]:
bureau_balance_02 = spark.sql(f'''
SELECT
  SK_ID_BUREAU as SK_ID_BUREAU_bureau_balance,
  {resultados_str}
FROM
  bureau_balance_01
GROUP BY
  SK_ID_BUREAU
''')

bureau_balance_02.createOrReplaceTempView("bureau_balance_02")
bureau_balance_02.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

817395

In [26]:
print(len(bureau_balance_02.columns))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

161

## **Salvando tabela no diretório do S3.**

In [29]:
# Save to parquet
path_output = os.path.join('s3://',S3_DATA_OUTPUT_PATH,output)
print(path_output)
bureau_balance_02.coalesce(1).write.\
parquet(path_output,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/01-Bureau/abt_bureau_balance

In [1]:
# abt_bureau_balance=sqlContext.read.format("parquet").load('s3://pod-academy-analise-de-credito-para-fintech/feature-engineering/01-Bureau/abt_bureau_balance')
# abt_bureau_balance.registerTempTable("abt_bureau_balance")

Starting Spark application


The code failed because of a fatal error:
	Session 8 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


In [34]:
# abt_bureau_balance.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+---------------

In [ ]:
# +---------------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+
# |SK_ID_BUREAU_bureau_balance|sum_month_stat_3|sum_U3M_month_stat_3|sum_U6M_month_stat_3|sum_U9M_month_stat_3|sum_U12M_month_stat_3|sum_month_stat_0|sum_U3M_month_stat_0|sum_U6M_month_stat_0|sum_U9M_month_stat_0|sum_U12M_month_stat_0|sum_month_stat_c|sum_U3M_month_stat_c|sum_U6M_month_stat_c|sum_U9M_month_stat_c|sum_U12M_month_stat_c|sum_month_stat_x|sum_U3M_month_stat_x|sum_U6M_month_stat_x|sum_U9M_month_stat_x|sum_U12M_month_stat_x|sum_month_stat_1|sum_U3M_month_stat_1|sum_U6M_month_stat_1|sum_U9M_month_stat_1|sum_U12M_month_stat_1|sum_month_stat_5|sum_U3M_month_stat_5|sum_U6M_month_stat_5|sum_U9M_month_stat_5|sum_U12M_month_stat_5|sum_month_stat_4|sum_U3M_month_stat_4|sum_U6M_month_stat_4|sum_U9M_month_stat_4|sum_U12M_month_stat_4|sum_month_stat_2|sum_U3M_month_stat_2|sum_U6M_month_stat_2|sum_U9M_month_stat_2|sum_U12M_month_stat_2|max_month_stat_3|max_U3M_month_stat_3|max_U6M_month_stat_3|max_U9M_month_stat_3|max_U12M_month_stat_3|max_month_stat_0|max_U3M_month_stat_0|max_U6M_month_stat_0|max_U9M_month_stat_0|max_U12M_month_stat_0|max_month_stat_c|max_U3M_month_stat_c|max_U6M_month_stat_c|max_U9M_month_stat_c|max_U12M_month_stat_c|max_month_stat_x|max_U3M_month_stat_x|max_U6M_month_stat_x|max_U9M_month_stat_x|max_U12M_month_stat_x|max_month_stat_1|max_U3M_month_stat_1|max_U6M_month_stat_1|max_U9M_month_stat_1|max_U12M_month_stat_1|max_month_stat_5|max_U3M_month_stat_5|max_U6M_month_stat_5|max_U9M_month_stat_5|max_U12M_month_stat_5|max_month_stat_4|max_U3M_month_stat_4|max_U6M_month_stat_4|max_U9M_month_stat_4|max_U12M_month_stat_4|max_month_stat_2|max_U3M_month_stat_2|max_U6M_month_stat_2|max_U9M_month_stat_2|max_U12M_month_stat_2|min_month_stat_3|min_U3M_month_stat_3|min_U6M_month_stat_3|min_U9M_month_stat_3|min_U12M_month_stat_3|min_month_stat_0|min_U3M_month_stat_0|min_U6M_month_stat_0|min_U9M_month_stat_0|min_U12M_month_stat_0|min_month_stat_c|min_U3M_month_stat_c|min_U6M_month_stat_c|min_U9M_month_stat_c|min_U12M_month_stat_c|min_month_stat_x|min_U3M_month_stat_x|min_U6M_month_stat_x|min_U9M_month_stat_x|min_U12M_month_stat_x|min_month_stat_1|min_U3M_month_stat_1|min_U6M_month_stat_1|min_U9M_month_stat_1|min_U12M_month_stat_1|min_month_stat_5|min_U3M_month_stat_5|min_U6M_month_stat_5|min_U9M_month_stat_5|min_U12M_month_stat_5|min_month_stat_4|min_U3M_month_stat_4|min_U6M_month_stat_4|min_U9M_month_stat_4|min_U12M_month_stat_4|min_month_stat_2|min_U3M_month_stat_2|min_U6M_month_stat_2|min_U9M_month_stat_2|min_U12M_month_stat_2|avg_month_stat_3|avg_U3M_month_stat_3|avg_U6M_month_stat_3|avg_U9M_month_stat_3|avg_U12M_month_stat_3|avg_month_stat_0|avg_U3M_month_stat_0|avg_U6M_month_stat_0|avg_U9M_month_stat_0|avg_U12M_month_stat_0|avg_month_stat_c|avg_U3M_month_stat_c|avg_U6M_month_stat_c|avg_U9M_month_stat_c|avg_U12M_month_stat_c|avg_month_stat_x|avg_U3M_month_stat_x|avg_U6M_month_stat_x|avg_U9M_month_stat_x|avg_U12M_month_stat_x|avg_month_stat_1|avg_U3M_month_stat_1|avg_U6M_month_stat_1|avg_U9M_month_stat_1|avg_U12M_month_stat_1|avg_month_stat_5|avg_U3M_month_stat_5|avg_U6M_month_stat_5|avg_U9M_month_stat_5|avg_U12M_month_stat_5|avg_month_stat_4|avg_U3M_month_stat_4|avg_U6M_month_stat_4|avg_U9M_month_stat_4|avg_U12M_month_stat_4|avg_month_stat_2|avg_U3M_month_stat_2|avg_U6M_month_stat_2|avg_U9M_month_stat_2|avg_U12M_month_stat_2|
# +---------------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+
# |5001757                    |null            |null                |null                |null                |null                 |-117.0          |-3.0                |-14.0               |-14.0               |-51.0                |null            |null                |null                |null                |null                 |-90.0           |null                |-4.0                |-4.0                |-24.0                |-3.0            |-3.0                |-3.0                |-3.0                |-3.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |0.0             |0.0                 |0.0                 |0.0                 |0.0                  |null            |null                |null                |null                |null                 |-9.0            |null                |-4.0                |-4.0                |-9.0                 |-3.0            |-3.0                |-3.0                |-3.0                |-3.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-1.0            |-1.0                |-1.0                |-1.0                |-1.0                 |null            |null                |null                |null                |null                 |-11.0           |null                |-4.0                |-4.0                |-11.0                |-3.0            |-3.0                |-3.0                |-3.0                |-3.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-9.0            |-1.0                |-2.8                |-2.8                |-5.67                |null            |null                |null                |null                |null                 |-12.86          |null                |-4.0                |-4.0                |-8.0                 |-3.0            |-3.0                |-3.0                |-3.0                |-3.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |
# |5001758                    |null            |null                |null                |null                |null                 |-828.0          |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-253.0          |-6.0                |-21.0               |-21.0               |-78.0                |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-46.0           |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |0.0             |0.0                 |0.0                 |0.0                 |0.0                  |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-23.0           |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-1.0            |-1.0                |-1.0                |-1.0                |-1.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-34.5           |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-11.0           |-1.5                |-3.0                |-3.0                |-6.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |
# |5001765                    |null            |null                |null                |null                |null                 |-171.0          |null                |null                |null                |null                 |-325.0          |-6.0                |-21.0               |-21.0               |-78.0                |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-31.0           |null                |null                |null                |null                 |0.0             |0.0                 |0.0                 |0.0                 |0.0                  |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-26.0           |null                |null                |null                |null                 |-1.0            |-1.0                |-1.0                |-1.0                |-1.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-28.5           |null                |null                |null                |null                 |-12.5           |-1.5                |-3.0                |-3.0                |-6.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |
# |5001767                    |null            |null                |null                |null                |null                 |-27.0           |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-78.0           |-6.0                |-21.0               |-21.0               |-78.0                |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-14.0           |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |0.0             |0.0                 |0.0                 |0.0                 |0.0                  |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-13.0           |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-1.0            |-1.0                |-1.0                |-1.0                |-1.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-13.5           |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-6.0            |-1.5                |-3.0                |-3.0                |-6.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |
# |5001770                    |null            |null                |null                |null                |null                 |-121.0          |null                |-6.0                |-6.0                |-63.0                |null            |null                |null                |null                |null                 |-15.0           |-6.0                |-15.0               |-15.0               |-15.0                |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-9.0            |null                |-6.0                |-6.0                |-9.0                 |null            |null                |null                |null                |null                 |0.0             |0.0                 |0.0                 |0.0                 |0.0                  |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-10.0           |null                |-6.0                |-6.0                |-10.0                |null            |null                |null                |null                |null                 |-1.0            |-1.0                |-1.0                |-1.0                |-1.0                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |-11.0           |null                |-6.0                |-6.0                |-9.0                 |null            |null                |null                |null                |null                 |-2.5            |-1.5                |-2.5                |-2.5                |-2.5                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |null            |null                |null                |null                |null                 |
# +---------------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+----------------+--------------------+--------------------+--------------------+---------------------+